<a href="https://colab.research.google.com/github/leandromrtk/100ProjetosFrontEnd/blob/main/Pr%C3%A1tica_2_Deep_Learning_CNN_Classificador_de_imagens_de_animais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **Prática: Reconhecimento de Imagens com Redes Neurais Convolucionais**

Problema: Dada uma imagem, é um cachorro ou um gato?

Assim como precisamos de dados históricos sobre clientes para que um algoritmo de Machine learming aprenda as regras para ter um crédito aprovado ou não (problema de classificação), para resolver esse problema precisamos primeiramente de muitas imagens de cachorros e gatos, para poder treinar um algoritmo de Deep Learning. Usaremos, portanto, uma abordagem de aprendizagem supervisionada, onde apresentaremos ao algoritmo diversas imagens, devidamente marcadas como sendo imagens de cães e gatos e então treinaremos o algoritmo. Ao final do treinamento, teremos um modelo que poderá receber novas imagens (desta vez não marcadas previamente) e então o modelo deverá ser capaz de classificar como sendo imagem de cão ou gato.

Solução: Usar uma Rede Neural Convolucional para aprender recursos de imagens e assim prever se uma imagem contém um cachorro ou um gato.

## **Definição dos Dados**


- Usamos os dados de treino para treinar o algoritmo e então criar o modelo preditivo.
- Usamos os dados de validação, para avaliar o modelo durante o treinamento.
- Usamos os dados de teste para validar a performance do modelo já treinado, ou seja, apresentamos ao modelo dados que ele não viu durante o treinamento, a fim de garantir que ele é capaz de fazer previsões.

Fonte: https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification

In [ ]:
import kagglehub

# Download latest version
caminho = kagglehub.dataset_download("samuelcortinhas/cats-and-dogs-image-classification")

print("Path to dataset files:", caminho)

Using Colab cache for faster access to the 'cats-and-dogs-image-classification' dataset.
Path to dataset files: /kaggle/input/cats-and-dogs-image-classification


In [ ]:
# Variáveis básicas
pasta_principal = caminho
pasta_treino = caminho + "/train"
pasta_testes = caminho + "/test"

import os

# Verifica se as pastas existem
print("Pasta train existe:", os.path.exists(pasta_treino))
print("Pasta test existe:", os.path.exists(pasta_testes))

# Lista o que tem dentro de treino
if os.path.exists(pasta_treino):
    print("Conteúdo de train:", os.listdir(pasta_treino))


In [ ]:
from IPython.display import Image, display

# Mostrar uma imagem específica
caminho_imagem = os.path.join(pasta_principal, "train", "dogs", "dog_102.jpg")

# Verificar se o arquivo existe
if os.path.exists(caminho_imagem):
    display(Image(filename=caminho_imagem, width=300))
else:
    print("Arquivo não encontrado: ",caminho_imagem)

## **Processamento de dados**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation

train_dir = "/kaggle/input/cats-and-dogs-image-classification/train"
test_dir = "/kaggle/input/cats-and-dogs-image-classification/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  #20 % PARA VALIDAÇÃO
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image_size = (128, 128)   # redefinir  o tamanho das imagens
batch_size = 32

#DADOS DE TREINO
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"    #80% TREINO
)

#DADOS DE VALIDAÇÃO = USAR DURANTE O TREINO
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"   #20% VALIDAÇÃO
)

#DADOS DE TESTE = PREDIÇÃO EM DADOS/IMAGENS NÃO USADAS NO TREINO
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)

In [ ]:
print("Mapeamento de classes:")
print(train_gen.class_indices)
print("Classes:", train_gen.classes)
print("Número de amostras:", train_gen.samples)

## **Construindo a Rede Neural Convolucional**


In [ ]:
from tensorflow.keras import layers, models

# 1. Define um simples CNN

# Modelo sequencial: as camadas são empilhadas em sequência
modelo_classificador = models.Sequential([

    # Primeira camada de convolução:
    # 32 filtros de tamanho 3x3, função de ativação ReLU.
    # A entrada são imagens 128x128 com 3 canais (RGB).
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),

    # Primeira camada de pooling:
    # Reduz a dimensão da imagem (2x2) mantendo as principais características.
    layers.MaxPooling2D((2,2)),

    # Segunda camada de convolução:
    # Aumenta o número de filtros para 64, aprendendo padrões mais complexos.
    layers.Conv2D(64, (3,3), activation='relu'),

    # Segunda camada de pooling:
    # Reduz novamente a dimensionalidade.
    layers.MaxPooling2D((2,2)),

    # Terceira camada de convolução:
    # 128 filtros para capturar detalhes mais profundos da imagem.
    layers.Conv2D(128, (3,3), activation='relu'),

    # Terceira camada de pooling:
    # Reduz a dimensionalidade final antes da parte densa.
    layers.MaxPooling2D((2,2)),

    # Flatten:
    # Transforma os mapas de características 2D em um vetor 1D.
    layers.Flatten(),

    # Camada densa totalmente conectada:
    # 128 neurônios, combina as características extraídas.
    layers.Dense(128, activation='relu'),

    # Dropout:
    # Desativa 50% dos neurônios durante o treino para evitar overfitting.
    layers.Dropout(0.5),

    # Camada de saída:
    # Um único neurônio com ativação sigmoid para classificação binária (0 ou 1).
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# 2. Compila o modelo antes do treino:

modelo_classificador.compile(
    optimizer='adam',              # Otimizador Adam: ajusta automaticamente a taxa de aprendizado
    loss='binary_crossentropy',    # Função de perda usada em classificações binárias (0 ou 1)
    metrics=['accuracy']           # Métrica de avaliação: acurácia
)

# Mostra o resumo da arquitetura da rede (opcional)
# modelo_classificador.summary()

# Treinamento do modelo:
modelo_classificador.fit(
    train_gen,                     # Conjunto de treinamento (gerador de imagens)
    validation_data=val_gen,       # Conjunto de validação (usado para avaliar o desempenho)
    epochs=10                      # Número de vezes que o modelo enxerga todos os dados para aprender
)

## **Fazendo Previsões**

Vamos agora testar nosso modelo treinado com imagens que ele ainda não viu e que estão nos dados de teste.

Para cada imagem de teste, carregamos as imagens com as mesmas dimensões usadas nas imagens de treino. Na sequência convertemos as imagens em um array e expandimos as dimensões. Então apresentamos as imagens ao classificador treinado nos passos anteriores. Por fim, verificamos o resultado da previsão e emitimos a informação se a imagem é de um gato ou cachorro.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Função para prever imagem: Faz predição em uma única imagem usando o mesmo pré-processamento do treino

def prever_imagem(modelo, caminho_imagem):

    # 1. Carregar a imagem
    img = image.load_img(caminho_imagem, target_size=(128, 128))  # Mesmo tamanho
    display(Image(filename=caminho_imagem, width=300))  #mostrar a imagem original

    # 2. Converter para array e aplicar mesmo pré-processamento
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão do batch
    img_array /= 255.0  # Mesma normalização: 1./255

    # 3. Fazer predição
    predicao = modelo.predict(img_array)[0][0]

    # 4. Interpretar resultado
    if predicao > 0.5:
        resultado = "É um Cachorro"
        confianca = predicao
    else:
        resultado = "É um gato"
        confianca = 1 - predicao

    print("Resultado: ", resultado)
    print("Confiança: ", confianca)

    return predicao

In [ ]:
# USAR A FUNÇÃO:
caminho_imagem_predicao = os.path.join(pasta_principal, "test", "dogs", "dog_130.jpg")
caminho_imagem_predicao_2 = os.path.join(pasta_principal, "test", "cats", "cat_118.jpg")

if os.path.exists(caminho_imagem_predicao_2):
    prever_imagem(modelo_classificador, caminho_imagem_predicao_2)
else:
    print("Arquivo não encontrado:", caminho_imagem_predicao_2)